In [1]:
import os
import shutil
import re

In [2]:
os.getcwd()

'/data/SE-VAE/jupyter'

In [3]:
def my_filter(path):
    log_path = os.path.join('..', path, 'log.out')
    log_s = open(log_path, 'r').readlines()
    test_path = os.path.join('..', path, 'test.out')
    test_s = open(log_path, 'r').readlines()
    #if False:
        #print(log_path)
        #return False
    #elif s[4] != 'history_length: 40\n' and s[5] != 'forward_length: 160\n':
        #print(log_path)
        #return False
    #elif 'Error' in test_s[-2]:
        #print(log_path)
        #return False
    #else:
    return True
    
def generating_dir(base_dir, root_dir):
    ret = []
    for file in os.listdir(os.path.join(root_dir, base_dir)):
        if file == 'tmp':
            continue
        path = os.path.join(base_dir, file)
        if os.path.isdir(os.path.join(root_dir, path)):
            ret = ret + generating_dir(path, root_dir)
        elif file == 'log.out':                                                   #and os.path.exists(os.path.join(root_dir, base_dir, 'best.pth')):
            if my_filter(os.path.split(path)[0]):
                ret.append(os.path.split(path)[0])
            else:
                continue
    return ret

path_list = generating_dir('ckpt'+os.path.sep, '..'+os.path.sep)
print('\n'.join(path_list))
print('共%d个训练模型'%len(path_list))


ckpt/cstr/ct_True/final_vae_rnn/vaernn_ct_time=True,random_seed=0,sp=0.5,sp_even=True,train.batch_size=2048/2022-06-04_13-26-47
ckpt/cstr/ct_True/final_vae_rnn/vaernn_ct_time=True,sp=0.25,sp_even=True,train.batch_size=2048/2022-07-04_15-46-27
ckpt/cstr/ct_True/final_vae_rnn/vaernn_ct_time=True,random_seed=0,sp=0.5,sp_even=False,train.batch_size=2048/2022-06-04_13-46-41
ckpt/cstr/ct_True/final_vae_rnn/vaernn_ct_time=True,sp=0.25,sp_even=False,train.batch_size=2048/2022-07-04_10-32-23
ckpt/cstr/ct_True/final_vae_rnn/vaernn_ct_time=True,sp=0.5,sp_even=False,train.batch_size=2048/2022-07-04_11-02-42
ckpt/cstr/ct_True/final_vae_rnn/vaernn_ct_time=True,random_seed=0,sp=0.25,sp_even=True,train.batch_size=2048/2022-06-04_12-40-17
ckpt/cstr/ct_True/final_vae_rnn/vaernn_ct_time=True,sp=1.0,sp_even=False,train.batch_size=2048/2022-07-04_23-34-06
ckpt/cstr/ct_True/final_vae_rnn/vaernn_ct_time=true,random_seed=0,sp=1.0,sp_even=true,train.batch_size=2048/2022-06-04_10-54-52
ckpt/cstr/ct_True/final_v

In [4]:
#检查log文件#可不可以直接找不存在best.pth的路径
error_log_list=[]                    #错误log文件路径集合
for t in range(len(path_list)):      #第t个临时列表中的路径
    error_log = False
    log_data = ''
    #print('t=',t)
    if not os.path.exists(os.path.join('..', path_list[t], 'best.pth')):
        error_log = True                   #存在error的标志
        error_line = "无模型"
    else:
        f = open('..'+os.path.sep+path_list[t]+os.path.sep+'test.out','r')
        log_data = f.readlines()
        f.close()
        for line in log_data[-100:]:
            if re.search('Error',line):
                error_log, error_line = True, line
                break
            if re.search('Early stopping at epoch',line):
                error_log = False
                break
            if re.search('Training finished', line):
                error_log = False
                break
        else:
            if len(log_data)<10:
                error_log, error_line = True, 'Not finished'
          
    if error_log==True:
        error_log_list.append(path_list[t])
        print('Error: ', path_list[t], error_line, '\n', 'Lines: %d' % len(log_data))
print('共 %d error log dir' % len(error_log_list))

Error:  ckpt/cstr/ct_True/final_rssm/rssm_ct_time=True,model.D=10,sp=0.5,sp_even=True,train.batch_size=2048/2022-07-04_02-24-43 无模型 
 Lines: 0
Error:  ckpt/cstr/ct_True/final_rssm/rssm_ct_time=True,model.D=5,sp=0.5,sp_even=True,train.batch_size=2048/2022-07-04_02-24-39 无模型 
 Lines: 0
Error:  ckpt/cstr/ct_True/final_rssm/rssm_ct_time=true,model.D=5,sp=1.0,sp_even=True,train.batch_size=1024/2022-05-31_04-44-59 无模型 
 Lines: 0
Error:  ckpt/cstr/ct_True/final_rssm/rssm_ct_time=true,model.D=5,sp=1.0,sp_even=True,train.batch_size=1024/2022-06-01_08-16-41 无模型 
 Lines: 0
Error:  ckpt/cstr/ct_True/final_rssm/rssm_ct_time=True,model.D=1,random_seed=0,sp=1.0,sp_even=True,train.batch_size=2048/2022-06-04_06-25-20 无模型 
 Lines: 0
Error:  ckpt/cstr/ct_True/final_rssm/rssm_ct_time=True,model.D=5,random_seed=0,sp=0.5,sp_even=False,train.batch_size=2048/2022-06-04_06-11-53 无模型 
 Lines: 0
Error:  ckpt/cstr/ct_True/final_rssm/rssm_ct_time=True,model.D=5,random_seed=0,sp=1.0,sp_even=True,train.batch_size=20

FileNotFoundError: [Errno 2] No such file or directory: '../ckpt/cstr/ct_True/final_ode_rssm/ode_rssm_ct_time=True,model.D=1,random_seed=0,sp=0.25,sp_even=True,train.batch_size=2048/2022-06-04_04-01-43/test.out'

In [19]:
def Delete_log(path):
    print("function have been triggered")
    print(path)
    shutil.rmtree(path)         #删除当前目录文件夹以及文件夹内的所有文件
    path=os.path.dirname(path)
    print(path)
    if not os.listdir(path):
        Delete_log(path)
    else:
        print("Delete accomplished")
        return 0
Delete_list=[]
if error_log_list:
    print("log文件报错路径如下：")
    for line in range(len(error_log_list)):
        Delete_list.append(error_log_list[line])
        print(Delete_list[line])
    ju=input("确定删除路径文件及其所在文件夹？ 请输入'yes' or 'no'")
    if ju=='yes':
        for line in range(len(Delete_list)):
            print(line)
            Delete_log('..'+os.path.sep+Delete_list[line])
    else:
        print("删除取消，保留路径文件")
else:
    print('未发现log文件报错')

未发现log文件报错


In [32]:
print('\n'.join(sorted(path_list, key=lambda x:x.split('/')[-1])))

ckpt/winding/ct_True/ode_merge/ode_rssm_merge_ct_time=True,sp=0.25,train.batch_size=2048/2022-05-15_05-15-25
ckpt/winding/ct_True/ode_merge/ode_rssm_merge_ct_time=True,sp=0.5,train.batch_size=2048/2022-05-15_05-38-00
ckpt/ib/ct_True/ode_rssm/ode_rssm_ct_time=True,sp=0.25,train.batch_size=2048/2022-05-15_05-49-34
ckpt/nl/ct_True/ode_rssm/ode_rssm_ct_time=True,sp=0.25,train.batch_size=2048/2022-05-15_05-50-07
ckpt/winding/ct_True/ode_rssm/ode_rssm_ct_time=True,sp=0.25,train.batch_size=2048/2022-05-15_06-25-27
ckpt/ib/ct_True/ode_rssm/ode_rssm_ct_time=True,sp=0.5,train.batch_size=2048/2022-05-15_06-33-30
ckpt/winding/ct_True/ode_rssm/ode_rssm_ct_time=True,sp=0.5,train.batch_size=2048/2022-05-15_07-02-34
ckpt/nl/ct_True/ode_rssm/ode_rssm_ct_time=True,sp=0.5,train.batch_size=2048/2022-05-15_07-03-35
ckpt/ib/ct_True/ode_rnn/ode_rnn_ct_time=True,sp=0.25,train.batch_size=2048/2022-05-15_07-47-12
ckpt/ib/ct_True/ode_rnn/ode_rnn_ct_time=True,sp=0.5,train.batch_size=2048/2022-05-15_08-44-05
ckpt/